In [2]:
import pandas as pd

df_train= pd.read_csv("cfg_advanced_features.csv")
df_test = pd.read_csv("cfg_advanced_features_test.csv")

In [3]:
meta_train= pd.read_csv("training_set_metadata.csv", sep=";")

In [4]:
meta_test=pd.read_excel("test_set_metadata_to_predict.xlsx")

In [5]:
df_features_train = df_train.rename(columns={'hash': 'name'})

# 2. R√©organiser df_features_train pour correspondre √† l'ordre de meta_train
# Supposons que 'name' est maintenant une colonne commune aux deux DataFrames
df_features_train = df_features_train.set_index('name')
df_features_train = df_features_train.reindex(index=meta_train['name'])
df_features_train = df_features_train.reset_index()

In [6]:
# V√©rification
are_identical = (df_features_train['name'].values == meta_train['name'].values).all()
print(f"Les DataFrames sont dans le m√™me ordre: {are_identical}")

Les DataFrames sont dans le m√™me ordre: True


In [7]:
X= df_features_train.drop(columns=['name'])

In [8]:
y=meta_train.drop(columns=["name"])

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(
            X, y.values, test_size=0.2, random_state=42
        )

In [11]:
# Supposons y est ton y_train sous forme numpy ou DataFrame
import numpy as np

# Par exemple : ne garder que les colonnes avec au moins 10 "1"
min_positive = 10
cols_to_keep = np.where((y_train == 1).sum(axis=0) >= min_positive)[0]
y_train2 = y_train[:, cols_to_keep]  # ou Data
y_val2= y_val[:, cols_to_keep]

In [15]:
# Seuil minimal d'√©chantillons positifs
min_positives = 10

# Filtrer les colonnes avec trop peu d'exemples positifs
filtered_columns = []
for col in y.columns:
    positive_count = y[col].sum()
    if positive_count >= min_positives:
        filtered_columns.append(col)
    else:
        print(f"Colonne {col} exclue: seulement {positive_count} positifs")

# Utiliser uniquement les colonnes filtr√©es
y_2 = y[filtered_columns]

Colonne anorganix exclue: seulement 5 positifs
Colonne apatch exclue: seulement 1 positifs
Colonne arm exclue: seulement 2 positifs
Colonne cexe exclue: seulement 1 positifs
Colonne check for Windows sandbox via genuine state exclue: seulement 1 positifs
Colonne check for Windows sandbox via process name exclue: seulement 2 positifs
Colonne check for protected handle exception exclue: seulement 8 positifs
Colonne check process job object exclue: seulement 4 positifs
Colonne checks-hostname exclue: seulement 2 positifs
Colonne contains-pe exclue: seulement 2 positifs
Colonne create new application domain in .NET exclue: seulement 1 positifs
Colonne create process on Linux exclue: seulement 3 positifs
Colonne create reverse shell on Linux exclue: seulement 1 positifs
Colonne create thread bypassing process freeze exclue: seulement 3 positifs
Colonne create virtual file system in .NET exclue: seulement 2 positifs
Colonne crypto exclue: seulement 7 positifs
Colonne decode data using Base64

In [17]:
from xgboost import XGBClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold
import numpy as np

import warnings
warnings.filterwarnings('ignore')


cv = KFold(n_splits=5, shuffle=True, random_state=42)
custom_threshold = 0.4
all_column_scores = {}

for col_idx, col_name in enumerate(y_2.columns):  # y doit √™tre un DataFrame
    print(f"Traitement de la colonne : {col_name}")
    col_scores = []

    for train_idx, test_idx in cv.split(X):
        X_train_fold, X_test_fold = X.iloc[train_idx], X.iloc[test_idx]
        y_train_fold, y_test_fold = y_2.iloc[train_idx, col_idx], y_2.iloc[test_idx, col_idx]
        
        model = XGBClassifier(
            tree_method='gpu_hist', 
            predictor='gpu_predictor', 
            use_label_encoder=False, 
            eval_metric='logloss'
        )
        model.fit(X_train_fold, y_train_fold)
        y_proba = model.predict_proba(X_test_fold)

        # Proba pour la classe 1 uniquement (classe positive)
        y_pred = (y_proba[:, 1] > custom_threshold).astype(int)
        
        score = f1_score(y_test_fold, y_pred, zero_division=0)
        col_scores.append(score)
    
    all_column_scores[col_name] = np.mean(col_scores)

# R√©sum√©
for col, score in all_column_scores.items():
    print(f"{col}: F1-macro = {score:.4f}")


Traitement de la colonne : 64-bit execution via heavens gate


Traitement de la colonne : 64bits
Traitement de la colonne : PEB access
Traitement de la colonne : accept command line arguments
Traitement de la colonne : access the Windows event log
Traitement de la colonne : act as TCP client
Traitement de la colonne : allocate RW memory
Traitement de la colonne : allocate RWX memory
Traitement de la colonne : allocate memory
Traitement de la colonne : allocate or change RW memory
Traitement de la colonne : allocate or change RWX memory
Traitement de la colonne : allocate thread local storage
Traitement de la colonne : android
Traitement de la colonne : apk
Traitement de la colonne : armadillo
Traitement de la colonne : aspack
Traitement de la colonne : asprotect
Traitement de la colonne : assembly
Traitement de la colonne : attach user process memory
Traitement de la colonne : attachment
Traitement de la colonne : authenticate HMAC
Traitement de la colonne : block operations on executable memory pages using Arbitrary Code Guard
Traitement de la co

In [62]:
from sklearn.model_selection import cross_val_predict, KFold
from sklearn.metrics import f1_score
from xgboost import XGBClassifier
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

cv = KFold(n_splits=5, shuffle=True, random_state=42)
threshold = 0.5
f1_scores_summary = []

for col in y.columns:
    print(f"üîç –û–±—Ä–∞–±–æ—Ç–∫–∞: {col}")

    model = XGBClassifier(
        tree_method='gpu_hist',
        predictor='gpu_predictor',
        use_label_encoder=False,
        eval_metric='logloss'
    )

    try:
        y_pred_proba = cross_val_predict(model, X, y[col], cv=cv, method='predict_proba')[:, 1]
        y_pred = (y_pred_proba > threshold).astype(int)
        f1 = f1_score(y[col], y_pred, zero_division=0)
        f1_scores_summary.append({'label': col, 'f1_macro': f1})
    except Exception as e:
        print(f"‚ùå –ü—Ä–æ–±–ª–µ–º–∞ —Å {col}: {e}")
        f1_scores_summary.append({'label': col, 'f1_macro': 0.0})

# –†–µ–∑—É–ª—å—Ç–∞—Ç—ã –≤ DataFrame
f1_df = pd.DataFrame(f1_scores_summary)
f1_df_sorted = f1_df.sort_values(by='f1_macro', ascending=False)
print(f1_df_sorted)

print(f"F1 macro : {f1_df['f1_macro'].mean()}")

# –û—Å—Ç–∞–≤–∏–º —Ç–æ–ª—å–∫–æ —Ö–æ—Ä–æ—à–∏–µ
threshold_keep = 0.6
good_labels = f1_df_sorted[f1_df_sorted['f1_macro'] > threshold_keep]['label'].tolist()
print(f"\n‚úÖ –û—Å—Ç–∞–≤–ª—è–µ–º {len(good_labels)} –º–µ—Ç–æ–∫ –∏–∑ {len(y.columns)}")

# –û—Ç—Ñ–∏–ª—å—Ç—Ä–æ–≤–∞–Ω–Ω–∞—è –º–∞—Ç—Ä–∏—Ü–∞ y
y_filtered = y[good_labels]


üîç –û–±—Ä–∞–±–æ—Ç–∫–∞: 64-bit execution via heavens gate
üîç –û–±—Ä–∞–±–æ—Ç–∫–∞: 64bits
üîç –û–±—Ä–∞–±–æ—Ç–∫–∞: PEB access
üîç –û–±—Ä–∞–±–æ—Ç–∫–∞: accept command line arguments
üîç –û–±—Ä–∞–±–æ—Ç–∫–∞: access the Windows event log
üîç –û–±—Ä–∞–±–æ—Ç–∫–∞: act as TCP client
üîç –û–±—Ä–∞–±–æ—Ç–∫–∞: allocate RW memory
üîç –û–±—Ä–∞–±–æ—Ç–∫–∞: allocate RWX memory
üîç –û–±—Ä–∞–±–æ—Ç–∫–∞: allocate memory
üîç –û–±—Ä–∞–±–æ—Ç–∫–∞: allocate or change RW memory


Exception ignored on calling ctypes callback function: <bound method DataIter._next_wrapper of <xgboost.data.SingleBatchInternalIter object at 0x7208ec9ac0a0>>
Traceback (most recent call last):
  File "/root/sdc/.venv/lib/python3.10/site-packages/xgboost/core.py", line 585, in _next_wrapper
    def _next_wrapper(self, this: None) -> int:  # pylint: disable=unused-argument
KeyboardInterrupt: 


: 

In [11]:
from sklearn.model_selection import cross_val_predict
from xgboost import XGBClassifier

# –°–∫–æ–ø–∏—Ä—É–µ–º X, —á—Ç–æ–±—ã –Ω–µ –∏—Å–ø–æ—Ä—Ç–∏—Ç—å –æ—Ä–∏–≥–∏–Ω–∞–ª
X_extended = X.copy()

print("üì• –î–æ–±–∞–≤–ª—è–µ–º –ø—Ä–µ–¥—Å–∫–∞–∑–∞–Ω–∏—è –º–µ—Ç–æ–∫ –∏–∑ y_filtered –≤ X...")

for col in y_filtered.columns:
    print(f"üîÑ –ü—Ä–µ–¥—Å–∫–∞–∑–∞–Ω–∏–µ –¥–ª—è: {col}")
    
    model = XGBClassifier(
        tree_method='gpu_hist',
        predictor='gpu_predictor',
        use_label_encoder=False,
        eval_metric='logloss'
    )

    try:
        y_pred = cross_val_predict(model, X, y_filtered[col], cv=5, method='predict')
        X_extended[f"pred_{col}"] = y_pred  # –¥–æ–±–∞–≤–ª—è–µ–º –≤–µ—Ä–æ—è—Ç–Ω–æ—Å—Ç–∏ –≤ X
    except Exception as e:
        print(f"‚ùå –û—à–∏–±–∫–∞ –ø—Ä–∏ –æ–±—Ä–∞–±–æ—Ç–∫–µ {col}: {e}")
        continue


üì• –î–æ–±–∞–≤–ª—è–µ–º –ø—Ä–µ–¥—Å–∫–∞–∑–∞–Ω–∏—è –º–µ—Ç–æ–∫ –∏–∑ y_filtered –≤ X...
üîÑ –ü—Ä–µ–¥—Å–∫–∞–∑–∞–Ω–∏–µ –¥–ª—è: android
üîÑ –ü—Ä–µ–¥—Å–∫–∞–∑–∞–Ω–∏–µ –¥–ª—è: assembly
üîÑ –ü—Ä–µ–¥—Å–∫–∞–∑–∞–Ω–∏–µ –¥–ª—è: apk
üîÑ –ü—Ä–µ–¥—Å–∫–∞–∑–∞–Ω–∏–µ –¥–ª—è: 64bits
üîÑ –ü—Ä–µ–¥—Å–∫–∞–∑–∞–Ω–∏–µ –¥–ª—è: peexe
üîÑ –ü—Ä–µ–¥—Å–∫–∞–∑–∞–Ω–∏–µ –¥–ª—è: reflection
üîÑ –ü—Ä–µ–¥—Å–∫–∞–∑–∞–Ω–∏–µ –¥–ª—è: diminisher
üîÑ –ü—Ä–µ–¥—Å–∫–∞–∑–∞–Ω–∏–µ –¥–ª—è: manipulate CD-ROM drive
üîÑ –ü—Ä–µ–¥—Å–∫–∞–∑–∞–Ω–∏–µ –¥–ª—è: obfuscated with KoiVM
üîÑ –ü—Ä–µ–¥—Å–∫–∞–∑–∞–Ω–∏–µ –¥–ª—è: get COMSPEC environment variable
üîÑ –ü—Ä–µ–¥—Å–∫–∞–∑–∞–Ω–∏–µ –¥–ª—è: bypass Windows File Protection
üîÑ –ü—Ä–µ–¥—Å–∫–∞–∑–∞–Ω–∏–µ –¥–ª—è: encode data using XOR
üîÑ –ü—Ä–µ–¥—Å–∫–∞–∑–∞–Ω–∏–µ –¥–ª—è: delay execution
üîÑ –ü—Ä–µ–¥—Å–∫–∞–∑–∞–Ω–∏–µ –¥–ª—è: packed with petite
üîÑ –ü—Ä–µ–¥—Å–∫–∞–∑–∞–Ω–∏–µ –¥–ª—è: create or open file
üîÑ –ü—Ä–µ–¥—Å–∫–∞–∑–∞–Ω–∏–µ –¥–ª—è: send ICMP echo request
üîÑ –ü—Ä–µ–¥—Å–∫–∞–∑–∞–Ω–∏–µ –¥–ª—è: obfu

üîÑ –ü—Ä–µ–¥—Å–∫–∞–∑–∞–Ω–∏–µ –¥–ª—è: create TCP socket
üîÑ –ü—Ä–µ–¥—Å–∫–∞–∑–∞–Ω–∏–µ –¥–ª—è: hash data using fnv
üîÑ –ü—Ä–µ–¥—Å–∫–∞–∑–∞–Ω–∏–µ –¥–ª—è: query or enumerate registry key
üîÑ –ü—Ä–µ–¥—Å–∫–∞–∑–∞–Ω–∏–µ –¥–ª—è: connect TCP socket
üîÑ –ü—Ä–µ–¥—Å–∫–∞–∑–∞–Ω–∏–µ –¥–ª—è: pedll
üîÑ –ü—Ä–µ–¥—Å–∫–∞–∑–∞–Ω–∏–µ –¥–ª—è: run as service
üîÑ –ü—Ä–µ–¥—Å–∫–∞–∑–∞–Ω–∏–µ –¥–ª—è: delete registry key
üîÑ –ü—Ä–µ–¥—Å–∫–∞–∑–∞–Ω–∏–µ –¥–ª—è: detect-debug-environment
üîÑ –ü—Ä–µ–¥—Å–∫–∞–∑–∞–Ω–∏–µ –¥–ª—è: create UDP socket
üîÑ –ü—Ä–µ–¥—Å–∫–∞–∑–∞–Ω–∏–µ –¥–ª—è: check for unmoving mouse cursor
üîÑ –ü—Ä–µ–¥—Å–∫–∞–∑–∞–Ω–∏–µ –¥–ª—è: telephony
üîÑ –ü—Ä–µ–¥—Å–∫–∞–∑–∞–Ω–∏–µ –¥–ª—è: decompress data using aPLib
üîÑ –ü—Ä–µ–¥—Å–∫–∞–∑–∞–Ω–∏–µ –¥–ª—è: execute anti-debugging instructions
üîÑ –ü—Ä–µ–¥—Å–∫–∞–∑–∞–Ω–∏–µ –¥–ª—è: write and execute a file
üîÑ –ü—Ä–µ–¥—Å–∫–∞–∑–∞–Ω–∏–µ –¥–ª—è: get disk information
üîÑ –ü—Ä–µ–¥—Å–∫–∞–∑–∞–Ω–∏–µ –¥–ª—è: hash data with MD5
üîÑ –ü—Ä–µ–¥—Å–∫–∞–∑–∞–Ω–∏–µ –¥–ª—è: encrypt da

In [12]:
y_remaining = y.drop(columns=y_filtered.columns)


In [2]:
y_remaining= pd.read_csv("y_rem.csv")
X_extended=pd.read_csv("X_extended.csv")

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_predict, KFold
from sklearn.metrics import f1_score
import pandas as pd
import numpy as np

cv = KFold(n_splits=5, shuffle=True, random_state=42)
threshold = 0.38
f1_scores_xgboost = []

for col in y_remaining.columns:
    print(f"üöÄ XGBoost pour : {col}")

    model = XGBClassifier(
        tree_method='gpu_hist',
        predictor='gpu_predictor',
        use_label_encoder=False,
        eval_metric='logloss',
        random_state=42
    )

    try:
        y_pred_proba = cross_val_predict(model, X_extended, y_remaining[col], cv=cv, method='predict_proba')[:, 1]
        y_pred = (y_pred_proba > threshold).astype(int)

        f1 = f1_score(y_remaining[col], y_pred, zero_division=0)
        f1_scores_xgboost.append({'label': col, 'f1_macro': f1})
    except Exception as e:
        print(f"‚ùå Erreur avec {col} : {e}")
        f1_scores_xgboost.append({'label': col, 'f1_macro': 0.0})

# R√©sum√© en DataFrame
xgb_df = pd.DataFrame(f1_scores_xgboost).sort_values(by='f1_macro', ascending=False)

# F1 macro global
print(f"\nüéØ F1-score macro moyen (XGBoost + threshold {threshold}): {xgb_df['f1_macro'].mean():.4f}")

# Colonnes performantes
xgb_good = xgb_df[xgb_df['f1_macro'] > 0.65]['label'].tolist()
print(f"‚úÖ XGBoost a trouv√© {len(xgb_good)} colonnes avec F1 > 0.65")



üê± CatBoost pour : 64-bit execution via heavens gate
üê± CatBoost pour : access the Windows event log
üê± CatBoost pour : act as TCP client
üê± CatBoost pour : allocate RW memory
üê± CatBoost pour : allocate RWX memory
üê± CatBoost pour : allocate or change RW memory
üê± CatBoost pour : allocate or change RWX memory
üê± CatBoost pour : anorganix
üê± CatBoost pour : apatch
‚ùå Erreur avec apatch : catboost/private/libs/target/target_converter.cpp:404: Target contains only one unique value
üê± CatBoost pour : arm
üê± CatBoost pour : armadillo
üê± CatBoost pour : aspack
üê± CatBoost pour : asprotect
üê± CatBoost pour : attach user process memory
üê± CatBoost pour : attachment
üê± CatBoost pour : block operations on executable memory pages using Arbitrary Code Guard
üê± CatBoost pour : bobsoft
üê± CatBoost pour : calls-wmi
üê± CatBoost pour : capture screenshot in Go
üê± CatBoost pour : cexe
‚ùå Erreur avec cexe : catboost/private/libs/target/target_converter.cpp:404: 

In [6]:
catboost_df = pd.DataFrame(f1_scores_catboost).sort_values(by='f1_macro', ascending=False)

print(catboost_df["f1_macro"].mean())
# Colonnes avec bonne performance
catboost_good = catboost_df[catboost_df['f1_macro'] > 0.65]['label'].tolist()
print(f"‚úÖ CatBoost a trouv√© {len(catboost_good)} bonnes colonnes avec F1 > 0.7")


0.26701089113231874
‚úÖ CatBoost a trouv√© 3 bonnes colonnes avec F1 > 0.7


In [7]:
from catboost import CatBoostClassifier
from sklearn.model_selection import cross_val_predict

for col in catboost_good:
    print(f"‚ûï Pr√©diction (CatBoost) pour : {col}")
    model = CatBoostClassifier(
        iterations=300,
        learning_rate=0.05,
        depth=6,
        loss_function='Logloss',
        eval_metric='F1',
        task_type='GPU',
        devices='0',
        verbose=0,
        random_seed=42,
        auto_class_weights='Balanced'
    )

    try:
        y_pred = cross_val_predict(model, X_extended, y_remaining[col], cv=5, method='predict')
        X_extended[f"catboost_pred_{col}"] = y_pred
    except Exception as e:
        print(f"‚ùå Erreur sur {col} : {e}")


‚ûï Pr√©diction (CatBoost) pour : check for protected handle exception
‚ûï Pr√©diction (CatBoost) pour : patch process command line
‚ûï Pr√©diction (CatBoost) pour : packed with nspack


In [9]:
y_remaining = y_remaining.drop(columns=catboost_good)


In [10]:
from sklearn.ensemble import IsolationForest

X_extended_anomalies = X_extended.copy()
anomaly_labels_kept = []

for col in y_remaining.columns:
    print(f"üîç Anomalie (IsolationForest) pour : {col}")
    positive_idx = y_remaining[col] == 1

    if positive_idx.sum() < 5:
        print(f"‚õîÔ∏è Trop peu de positifs ({positive_idx.sum()}), on skip")
        continue

    try:
        # On entra√Æne uniquement sur les positifs (1)
        model = IsolationForest(
            n_estimators=100,
            contamination=0.01,  # suppose qu‚Äôon cherche les plus rares
            random_state=42
        )
        model.fit(X_extended.loc[positive_idx])

        # On pr√©dit sur toutes les donn√©es
        preds = model.predict(X_extended)
        # IsolationForest renvoie : 1 (normal), -1 (anomaly)
        preds_binary = (preds == -1).astype(int)

        # Ajouter au X
        X_extended_anomalies[f"anomaly_pred_{col}"] = preds_binary
        anomaly_labels_kept.append(col)

    except Exception as e:
        print(f"‚ùå Erreur sur {col} : {e}")


üîç Anomalie (IsolationForest) pour : 64-bit execution via heavens gate
üîç Anomalie (IsolationForest) pour : access the Windows event log
üîç Anomalie (IsolationForest) pour : act as TCP client
üîç Anomalie (IsolationForest) pour : allocate RW memory
üîç Anomalie (IsolationForest) pour : allocate RWX memory
üîç Anomalie (IsolationForest) pour : allocate or change RW memory
üîç Anomalie (IsolationForest) pour : allocate or change RWX memory
üîç Anomalie (IsolationForest) pour : anorganix
üîç Anomalie (IsolationForest) pour : apatch
‚õîÔ∏è Trop peu de positifs (1), on skip
üîç Anomalie (IsolationForest) pour : arm
‚õîÔ∏è Trop peu de positifs (2), on skip
üîç Anomalie (IsolationForest) pour : armadillo
üîç Anomalie (IsolationForest) pour : aspack
üîç Anomalie (IsolationForest) pour : asprotect
üîç Anomalie (IsolationForest) pour : attach user process memory
üîç Anomalie (IsolationForest) pour : attachment
üîç Anomalie (IsolationForest) pour : block operations on executabl

/tmp/ipykernel_3333/847216270.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_extended_anomalies[f"anomaly_pred_{col}"] = preds_binary
/tmp/ipykernel_3333/847216270.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_extended_anomalies[f"anomaly_pred_{col}"] = preds_binary


üîç Anomalie (IsolationForest) pour : encrypt or decrypt via WinCrypt
üîç Anomalie (IsolationForest) pour : enigma


/tmp/ipykernel_3333/847216270.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_extended_anomalies[f"anomaly_pred_{col}"] = preds_binary
/tmp/ipykernel_3333/847216270.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_extended_anomalies[f"anomaly_pred_{col}"] = preds_binary


üîç Anomalie (IsolationForest) pour : enumerate PE sections
üîç Anomalie (IsolationForest) pour : enumerate PE sections in .NET


...

/tmp/ipykernel_3333/847216270.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_extended_anomalies[f"anomaly_pred_{col}"] = preds_binary
/tmp/ipykernel_3333/847216270.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_extended_anomalies[f"anomaly_pred_{col}"] = preds_binary


üîç Anomalie (IsolationForest) pour : validate payment card number using luhn algorithm with lookup table
‚õîÔ∏è Trop peu de positifs (1), on skip
üîç Anomalie (IsolationForest) pour : validate payment card number using luhn algorithm with no lookup table
üîç Anomalie (IsolationForest) pour : via-tor


/tmp/ipykernel_3333/847216270.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_extended_anomalies[f"anomaly_pred_{col}"] = preds_binary
/tmp/ipykernel_3333/847216270.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_extended_anomalies[f"anomaly_pred_{col}"] = preds_binary


üîç Anomalie (IsolationForest) pour : winzip
‚õîÔ∏è Trop peu de positifs (2), on skip
üîç Anomalie (IsolationForest) pour : wise
‚õîÔ∏è Trop peu de positifs (1), on skip
üîç Anomalie (IsolationForest) pour : worm
‚õîÔ∏è Trop peu de positifs (2), on skip
üîç Anomalie (IsolationForest) pour : write file on Linux
üîç Anomalie (IsolationForest) pour : xorcrypt
üîç Anomalie (IsolationForest) pour : yoda
‚õîÔ∏è Trop peu de positifs (1), on skip


/tmp/ipykernel_3333/847216270.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_extended_anomalies[f"anomaly_pred_{col}"] = preds_binary
/tmp/ipykernel_3333/847216270.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_extended_anomalies[f"anomaly_pred_{col}"] = preds_binary


In [45]:
from sklearn.metrics import f1_score
import pandas as pd

anomaly_f1_scores = []

for col in anomaly_labels_kept:
    y_true = y_remaining[col].values
    y_pred_random = np.random.choice([0, 1], size=n, p=[1/2, 1/2])
    score = f1_score(y_true, y_pred_random, zero_division=0)
    anomaly_f1_scores.append({'label': col, 'f1_macro': score})

# R√©sultats
anomaly_f1_df = pd.DataFrame(anomaly_f1_scores).sort_values(by='f1_macro', ascending=False)

# F1 global
print(anomaly_f1_df)
print(f"\nüéØ F1-score macro moyen (IsolationForest): {anomaly_f1_df['f1_macro'].mean():.4f}")


                                label  f1_macro
44                  checks-user-input  0.341614
239                            signed  0.307665
207  reference analysis tools strings  0.290050
154                       long-sleeps  0.281609
59                       create mutex  0.235842
..                                ...       ...
186          packed with y0da crypter  0.000342
7                           anorganix  0.000175
151              linked against aPLib  0.000175
256               write file on Linux  0.000174
144                          kkrunchy  0.000171

[258 rows x 2 columns]

üéØ F1-score macro moyen (IsolationForest): 0.0443


In [46]:

# R√©sultats
anomaly_f1_df = pd.DataFrame(anomaly_f1_scores).sort_values(by='f1_macro', ascending=False)

# F1 global
print(anomaly_f1_df)
print(f"\nüéØ F1-score macro moyen (IsolationForest): {anomaly_f1_df['f1_macro'].mean():.4f}")


                                label  f1_macro
44                  checks-user-input  0.341614
239                            signed  0.307665
207  reference analysis tools strings  0.290050
154                       long-sleeps  0.281609
59                       create mutex  0.235842
..                                ...       ...
186          packed with y0da crypter  0.000342
7                           anorganix  0.000175
151              linked against aPLib  0.000175
256               write file on Linux  0.000174
144                          kkrunchy  0.000171

[258 rows x 2 columns]

üéØ F1-score macro moyen (IsolationForest): 0.0443


In [47]:
top5 = anomaly_f1_df.head(5)

In [48]:
top5

label  f1_macro
44                  checks-user-input  0.341614
239                            signed  0.307665
207  reference analysis tools strings  0.290050
154                       long-sleeps  0.281609
59                       create mutex  0.235842

In [20]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import f1_score

cv = KFold(n_splits=5, shuffle=True, random_state=42)
threshold = 0.5
logreg_scores = []

for col in y.columns:
    print(f"üß™ Logistic Regression –¥–ª—è: {col}")
    model = LogisticRegression(max_iter=1000, class_weight='balanced')

    try:
        y_pred_proba = cross_val_predict(model, X, y[col], cv=cv, method='predict_proba')[:, 1]
        y_pred = (y_pred_proba > threshold).astype(int)
        f1 = f1_score(y[col], y_pred, zero_division=0)
        logreg_scores.append({'label': col, 'f1_macro': f1})
    except Exception as e:
        print(f"‚ùå –û—à–∏–±–∫–∞ –Ω–∞ {col}: {e}")
        logreg_scores.append({'label': col, 'f1_macro': 0.0})

logreg_df = pd.DataFrame(logreg_scores).sort_values(by='f1_macro', ascending=False)
print(logreg_df)

# –ù–∞–ø—Ä–∏–º–µ—Ä, —Å–æ—Ö—Ä–∞–Ω–∏–º –º–µ—Ç–∫–∏ –≥–¥–µ f1 > 0.5
logreg_good = logreg_df[logreg_df['f1_macro'] > 0.5]['label'].tolist()
print(f"üéØ Logistic Regression —Å–ø–∞—Å–ª–∞ –µ—â—ë {len(logreg_good)} –º–µ—Ç–æ–∫!")


üß™ Logistic Regression –¥–ª—è: 64-bit execution via heavens gate
üß™ Logistic Regression –¥–ª—è: 64bits
üß™ Logistic Regression –¥–ª—è: PEB access
üß™ Logistic Regression –¥–ª—è: accept command line arguments
üß™ Logistic Regression –¥–ª—è: access the Windows event log


KeyboardInterrupt: 

In [9]:
df_features_test = df_test.rename(columns={'hash': 'name'})

# 2. R√©organiser df_features_train pour correspondre √† l'ordre de meta_train
# Supposons que 'name' est maintenant une colonne commune aux deux DataFrames
df_features_test = df_features_test.set_index('name')
df_features_test = df_features_test.reindex(index=meta_test['name'])
df_features_test = df_features_test.reset_index()
# V√©rification
are_identical = (df_features_test['name'].values == meta_test['name'].values).all()
print(f"Les DataFrames sont dans le m√™me ordre: {are_identical}")

Les DataFrames sont dans le m√™me ordre: True


In [10]:
X_test=df_features_test.drop(columns=["name"])

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_predict, KFold
from sklearn.metrics import f1_score
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Param√®tres
cv = KFold(n_splits=5, shuffle=True, random_state=42)
thresholds = np.arange(0.5, 0.0, -0.05)

# Copies de travail
X_current = X.copy()
X_test_current = X_test.copy()
y_current = y.copy()

# R√©sultats
y_pred_train = pd.DataFrame(index=X.index)
y_pred_test = pd.DataFrame(index=X_test.index)

for t in thresholds:
    print(f"\nüéØ Threshold = {t:.2f}")
    step_cols = []

    for col in y_current.columns:
        print(f"üîç Colonne : {col}")
        model = XGBClassifier(
            tree_method='gpu_hist',
            predictor='gpu_predictor',
            use_label_encoder=False,
            eval_metric='logloss',
            random_state=42
        )

        try:
            y_proba = cross_val_predict(model, X_current, y_current[col], cv=cv, method='predict_proba')[:, 1]
            y_pred = (y_proba > t).astype(int)
            score = f1_score(y_current[col], y_pred, zero_division=0)

            if (t > 0.05 and score > 0.7) or (t == 0.05 and score > 0.3):
                print(f"‚úÖ Accept√© (F1 = {score:.4f})")
                step_cols.append(col)

                # Retrain sur full train pour test
                model.fit(X_current, y_current[col])
                y_proba_test = model.predict_proba(X_test_current)[:, 1]
                y_pred_test_ = (y_proba_test > t).astype(int)

                # Ajoute dans output + dans X comme nouvelle feature
                y_pred_train[col] = y_pred
                y_pred_test[col] = y_pred_test_
                X_current[f"pred_{col}"] = y_pred
                X_test_current[f"pred_{col}"] = y_pred_test_

        except Exception as e:
            print(f"‚ùå {col}: {e}")

    # Supprime les colonnes d√©j√† trait√©es
    y_current = y_current.drop(columns=step_cols)

# üé≤ G√©n√©ration al√©atoire pour les colonnes restantes
for col in y_current.columns:
    print(f"üé≤ Random pour : {col}")
    y_rand_train = np.random.choice([0, 1], size=len(X), p=[0.5, 0.5])
    y_rand_test = np.random.choice([0, 1], size=len(X_test), p=[0.5, 0.5])
    y_pred_train[col] = y_rand_train
    y_pred_test[col] = y_rand_test



üéØ Threshold = 0.50
üîç Colonne : 64-bit execution via heavens gate


üîç Colonne : 64bits
‚úÖ Accept√© (F1 = 0.9142)
üîç Colonne : PEB access
‚úÖ Accept√© (F1 = 0.7568)
üîç Colonne : accept command line arguments
üîç Colonne : access the Windows event log
üîç Colonne : act as TCP client
üîç Colonne : allocate RW memory
üîç Colonne : allocate RWX memory
üîç Colonne : allocate memory
üîç Colonne : allocate or change RW memory
üîç Colonne : allocate or change RWX memory
üîç Colonne : allocate thread local storage
üîç Colonne : android
‚úÖ Accept√© (F1 = 0.9624)
üîç Colonne : anorganix
üîç Colonne : apatch
‚ùå apatch: [22:52:51] /workspace/src/objective/./regression_loss.h:69: Check failed: base_score > 0.0f && base_score < 1.0f: base_score must be in (0,1) for logistic loss, got: 0
Stack trace:
  [bt] (0) /root/sdc/.venv/lib/python3.10/site-packages/xgboost/lib/libxgboost.so(+0x2a6acc) [0x7ed7d30a6acc]
  [bt] (1) /root/sdc/.venv/lib/python3.10/site-packages/xgboost/lib/libxgboost.so(+0xed8329) [0x7ed7d3cd8329]
  [bt] (2) /root/sdc/.venv/lib/p

In [14]:
y_pred_test["name"]=meta_test["name"]

In [15]:
y_pred_test

64bits  PEB access  android  apk  assembly  \
0          1           0        0    0         0   
1          0           0        0    0         0   
2          1           0        0    0         0   
3          0           0        0    0         0   
4          1           0        0    0         0   
...      ...         ...      ...  ...       ...   
2995       0           0        0    0         1   
2996       1           0        0    0         0   
2997       0           0        0    0         1   
2998       0           0        0    0         0   
2999       1           0        0    0         0   

      bypass Windows File Protection  calculate modulo 256 via x86 assembly  \
0                                  0                                      0   
1                                  0                                      0   
2                                  0                                      0   
3                                  0                                      0   
4                                  0                                      0   
...                              ...                                    ...   
2995                               0                                      0   
2996                               0                                      0   
2997                               0                                      0   
2998                               0                                      0   
2999                               0                                      0   

      capture screenshot  check for VM using instruction VPCEXT  \
0                      0                                      0   
1                      0                                      0   
2                      0                                      0   
3                      0                                      0   
4                      0                                      0   
...                  ...                                    ...   
2995                   0                                      0   
2996                   0                                      0   
2997                   0                                      0   
2998                   0                                      0   
2999                   0                                      0   

      check for hardware breakpoints  ...  winzip  wise  worm  \
0                                  0  ...       1     0     0   
1                                  0  ...       0     0     0   
2                                  0  ...       0     1     1   
3                                  0  ...       1     1     1   
4                                  0  ...       1     1     1   
...                              ...  ...     ...   ...   ...   
2995                               0  ...       1     1     0   
2996                               0  ...       0     1     1   
2997                               0  ...       1     1     0   
2998                               0  ...       0     1     0   
2999                               0  ...       0     1     1   

      write and execute a file  write clipboard data  write file on Linux  \
0                            1                     0                    0   
1                            1                     0                    0   
2                            1                     0                    1   
3                            1                     0                    1   
4                            0                     1                    0   
...                        ...                   ...                  ...   
2995                         1                     0                    1   
2996                         1                     0                    1   
2997                         1                     0                    0   
2998                         0                     0                    0   

In [18]:
# V√©rification de l'ordre des colonnes
if list(y_pred_test.columns) == list(meta_test.columns):
    print("‚úÖ L'ordre des colonnes est identique.")
else:
    print("‚ùå L'ordre des colonnes est diff√©rent.")
    # Affichage des diff√©rences pour investigation
    print("Colonnes y_pred_test :", list(y_pred_test.columns))
    print("Colonnes meta_test   :", list(meta_test.columns))

‚úÖ L'ordre des colonnes est identique.


In [17]:
# R√©ordonner les colonnes de y_pred_test pour qu'elles correspondent √† meta_test
y_pred_test = y_pred_test[meta_test.columns]


In [19]:
y_pred_test.to_excel("ebanutaya_hernya.xlsx", index=False)

In [20]:
y_pred_test

name  \
0     95ff2b3fd399984950293194a717d404bc4e9e3aa0296f...   
1     4216c07609bb5b89f32d9b559494848a0f5411d1e2d3cf...   
2     2518f84c015a5795bdb2597d580ab7df8e0bfa4b6543c6...   
3     243268c658456d9b8ec968c088c4f3c7cb976df92a4b99...   
4     533c09cb9b49c10494337d3eb7d2919c2c656b37f554fb...   
...                                                 ...   
2995  7f0a7ccc2abe273330885020f6dc5f116fe83535838b65...   
2996  a188ff24aec863479408cee54b337a2fce25b9372ba557...   
2997  11bf54ba134bfcf242e6c841ae56f44883cdf1b45fa24c...   
2998  7a8862b047f38a1f9ff37b6d1fa114f61bcb7ad6415bed...   
2999  5be7e227739f852689d1a0c7f4c42afe5e98a1090718e2...   

      64-bit execution via heavens gate  64bits  PEB access  \
0                                     1       1           0   
1                                     0       0           0   
2                                     1       1           0   
3                                     1       0           0   
4                                     0       1           0   
...                                 ...     ...         ...   
2995                                  1       0           0   
2996                                  0       1           0   
2997                                  0       0           0   
2998                                  0       0           0   
2999                                  1       1           0   

      accept command line arguments  access the Windows event log  \
0                                 1                             1   
1                                 0                             1   
2                                 1                             0   
3                                 1                             1   
4                                 1                             0   
...                             ...                           ...   
2995                              0                             0   
2996                              0                             0   
2997                              0                             1   
2998                              0                             1   
2999                              0                             0   

      act as TCP client  allocate RW memory  allocate RWX memory  \
0                     1                   0                    1   
1                     0                   1                    0   
2                     0                   0                    1   
3                     1                   1                    0   
4                     1                   1                    1   
...                 ...                 ...                  ...   
2995                  0                   1                    0   
2996                  1                   1                    0   
2997                  0                   0                    0   
2998                  0                   1                    0   
2999                  0                   0                    0   

      allocate memory  ...  winzip  wise  worm  write and execute a file  \
0                   0  ...       1     0     0                         1   
1                   0  ...       0     0     0                         1   
2                   1  ...       0     1     1                         1   
3                   1  ...       1     1     1                         1   
4                   0  ...       1     1     1                         0   
...               ...  ...     ...   ...   ...                       ...   
2995                0  ...       1     1     0                         1   
2996                1  ...       0     1     1                         1   
2997                1  ...       1     1     0                         1   
2998                0  ...       0     1     0                         0   
2999                1  ...       0     1     1                         0   

      write clipboard data

In [21]:
import numpy as np

# Copie de s√©curit√©
df = y_pred_test.copy()

# Parcourir chaque colonne
for col in df.columns:
    # Compter les valeurs 0 et 1
    counts = df[col].value_counts()
    
    # V√©rifier qu'il y a exactement 0 et 1, √† part √©gale
    if set(counts.index) == {0, 1} and counts[0] == counts[1]:
        print(f"üéØ Colonne √©quilibr√©e d√©tect√©e : {col}")
        
        # Obtenir les index des 1
        ones_indices = df[df[col] == 1].index
        
        # Calculer combien de 1 on veut garder
        n_keep = max(1, len(ones_indices) // 40)  # garde au moins 1
        
        # Choisir al√©atoirement lesquels garder
        keep_indices = np.random.choice(ones_indices, size=n_keep, replace=False)
        
        # Remettre tout √† 0
        df[col] = 0
        
        # Restaurer seulement les 1 s√©lectionn√©s
        df.loc[keep_indices, col] = 1

# df contient maintenant la version modifi√©e


üéØ Colonne √©quilibr√©e d√©tect√©e : dex
üéØ Colonne √©quilibr√©e d√©tect√©e : encrypt data using twofish
üéØ Colonne √©quilibr√©e d√©tect√©e : installstub
üéØ Colonne √©quilibr√©e d√©tect√©e : packed with TSULoader
üéØ Colonne √©quilibr√©e d√©tect√©e : reference anti-VM strings targeting VirtualPC
üéØ Colonne √©quilibr√©e d√©tect√©e : software-collection


In [22]:
y_current

64-bit execution via heavens gate  accept command line arguments  \
0                                      0                              0   
1                                      0                              0   
2                                      0                              0   
3                                      0                              0   
4                                      0                              0   
...                                  ...                            ...   
23097                                  0                              1   
23098                                  0                              0   
23099                                  0                              0   
23100                                  0                              0   
23101                                  0                              0   

       access the Windows event log  act as TCP client  allocate RW memory  \
0                                 0                  0                   0   
1                                 0                  0                   0   
2                                 0                  0                   0   
3                                 0                  0                   0   
4                                 0                  0                   0   
...                             ...                ...                 ...   
23097                             0                  0                   0   
23098                             0                  0                   0   
23099                             0                  0                   0   
23100                             0                  0                   0   
23101                             0                  0                   0   

       allocate RWX memory  allocate memory  allocate or change RW memory  \
0                        0                0                             0   
1                        0                0                             0   
2                        0                0                             0   
3                        0                0                             0   
4                        0                0                             0   
...                    ...              ...                           ...   
23097                    0                0                             0   
23098                    0                1                             1   
23099                    0                0                             0   
23100                    0                0                             0   
23101                    0                0                             0   

       allocate or change RWX memory  allocate thread local storage  ...  \
0                                  0                              0  ...   
1                                  0                              1  ...   
2                                  0                              0  ...   
3                                  0                              0  ...   
4                                  1                              0  ...   
...                              ...                            ...  ...   
23097                              0                              0  ...   
23098                              0                              0  ...   
23099                              0                              0  ...   
23100                              0                              0  ...   
23101                              0                              0  ...   

       via-tor  winzip  wise  worm  write and execute a file  \
0            0       0     0     0                         0   
1            0       0     0     0                         0   
2            0       0     0     0                         0   
3            0       0     0     0                         0   
4          

In [23]:
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_predict, KFold, train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

def train_multilabel_model(X, X_test, y, thresholds=np.arange(0.5, 0.0, -0.05), 
                          f1_threshold_high=0.7, f1_threshold_low=0.3, random_state=42):
    """
    Entra√Æne des mod√®les XGBoost pour chaque colonne cible en utilisant des seuils d√©croissants.
    Utilise les pr√©dictions comme features pour les mod√®les suivants.
    
    Args:
        X: DataFrame des features d'entra√Ænement
        X_test: DataFrame des features de test
        y: DataFrame des cibles multi-labels
        thresholds: Liste des seuils √† tester par ordre d√©croissant
        f1_threshold_high: Seuil F1 pour accepter une colonne avec un threshold √©lev√©
        f1_threshold_low: Seuil F1 minimal pour accepter une colonne avec le dernier threshold
        random_state: Seed pour la reproductibilit√©
    
    Returns:
        y_pred_train: Pr√©dictions sur l'ensemble d'entra√Ænement
        y_pred_test: Pr√©dictions sur l'ensemble de test
        metrics_df: DataFrame des m√©triques par colonne et threshold
    """
    # Validation cross-fold
    cv = KFold(n_splits=5, shuffle=True, random_state=random_state)
    
    # Copies de travail
    X_current = X.copy()
    X_test_current = X_test.copy()
    
    # R√©sultats
    y_pred_train = pd.DataFrame(index=X.index)
    y_pred_test = pd.DataFrame(index=X_test.index)
    
    # Suivi des m√©triques
    metrics = []
    
    # Colonnes restantes √† traiter
    remaining_cols = y.columns.tolist()
    
    # Parcours des seuils par ordre d√©croissant
    for t in thresholds:
        print(f"\nüéØ Threshold = {t:.2f}")
        cols_processed = []
        
        for col in remaining_cols:
            print(f"üîç Colonne : {col}")
            
            # Param√®tres XGBoost adapt√©s
            model = XGBClassifier(
                tree_method='gpu_hist',
                predictor='gpu_predictor',
                use_label_encoder=False,
                eval_metric='logloss',
                max_depth=6,
                learning_rate=0.1,
                n_estimators=100,
                subsample=0.8,
                colsample_bytree=0.8,
                random_state=random_state
            )
            
            try:
                # Pr√©diction par validation crois√©e
                y_proba = cross_val_predict(model, X_current, y[col], cv=cv, method='predict_proba')[:, 1]
                y_pred = (y_proba > t).astype(int)
                
                # Calcul des m√©triques
                f1 = f1_score(y[col], y_pred, zero_division=0)
                precision = precision_score(y[col], y_pred, zero_division=0)
                recall = recall_score(y[col], y_pred, zero_division=0)
                
                try:
                    auc = roc_auc_score(y[col], y_proba)
                except:
                    auc = 0.0
                
                # Enregistrement des m√©triques
                metrics.append({
                    'column': col,
                    'threshold': t,
                    'f1': f1,
                    'precision': precision,
                    'recall': recall,
                    'auc': auc
                })
                
                # D√©cision d'acceptation bas√©e sur le seuil F1
                f1_threshold = f1_threshold_low if t == thresholds[-1] else f1_threshold_high
                
                if f1 > f1_threshold:
                    print(f"‚úÖ Accept√© (F1={f1:.4f}, Prec={precision:.4f}, Rec={recall:.4f}, AUC={auc:.4f})")
                    
                    # Entrainement sur tout le jeu d'entra√Ænement
                    model.fit(X_current, y[col])
                    
                    # Pr√©diction sur le jeu de test
                    y_proba_test = model.predict_proba(X_test_current)[:, 1]
                    y_pred_test_ = (y_proba_test > t).astype(int)
                    
                    # Ajout dans les r√©sultats
                    y_pred_train[col] = y_pred
                    y_pred_test[col] = y_pred_test_
                    
                    # Ajout comme nouvelle feature
                    feature_name = f"pred_{col}_t{int(t*100)}"
                    X_current[feature_name] = y_pred
                    X_test_current[feature_name] = y_pred_test_
                    
                    # Marquer comme trait√©e
                    cols_processed.append(col)
                else:
                    print(f"‚ùå Rejet√© (F1={f1:.4f}, Prec={precision:.4f}, Rec={recall:.4f}, AUC={auc:.4f})")
            
            except Exception as e:
                print(f"‚ùå Erreur pour {col}: {e}")
        
        # Suppression des colonnes trait√©es
        for col in cols_processed:
            remaining_cols.remove(col)
            
        print(f"üìä Colonnes trait√©es avec seuil {t:.2f}: {len(cols_processed)}")
        print(f"üìã Colonnes restantes: {len(remaining_cols)}")
        
        # Si toutes les colonnes sont trait√©es, on sort
        if not remaining_cols:
            print("\n‚ú® Toutes les colonnes ont √©t√© trait√©es!")
            break
    
    # Traitement des colonnes restantes non-trait√©es
    if remaining_cols:
        print(f"\n‚ö†Ô∏è {len(remaining_cols)} colonnes n'ont pas atteint le seuil F1 minimum:")
        
        for col in remaining_cols:
            print(f"üîÆ Pr√©diction pour : {col}")
            
            # Au lieu de g√©n√©rer al√©atoirement, on pr√©dit la classe majoritaire
            majority_class = int(y[col].mean() >= 0.5)
            print(f"   Classe majoritaire : {majority_class} (distribution: {y[col].mean():.2f})")
            
            y_pred_train[col] = majority_class
            y_pred_test[col] = majority_class
    
    # Cr√©ation du DataFrame de m√©triques
    metrics_df = pd.DataFrame(metrics)
    
    return y_pred_train, y_pred_test, metrics_df

# Exemple d'utilisation
if __name__ == "__main__":
    # Supposons que X, X_test et y sont d√©j√† d√©finis
    # X: features d'entra√Ænement
    # X_test: features de test
    # y: cibles multi-labels
    
    # Configuration des seuils et des crit√®res
    thresholds = np.arange(0.5, 0.0, -0.05)
    f1_threshold_high = 0.7  # Seuil √©lev√© pour les premiers thresholds
    f1_threshold_low = 0.3   # Seuil minimum pour le dernier threshold
    
    # Entra√Ænement des mod√®les
    y_pred_train, y_pred_test, metrics_df = train_multilabel_model(
        X, X_test, y, 
        thresholds=thresholds,
        f1_threshold_high=f1_threshold_high,
        f1_threshold_low=f1_threshold_low
    )
    
    # Affichage des r√©sultats globaux
    print("\nüìà Performance globale:")
    f1_global = f1_score(y, y_pred_train, average='macro')
    print(f"F1-score macro moyen sur l'entra√Ænement: {f1_global:.4f}")
    
    # Sauvegarde des pr√©dictions
    y_pred_train.to_csv('predictions_train.csv')
    y_pred_test.to_csv('predictions_test.csv')


üéØ Threshold = 0.50
üîç Colonne : 64-bit execution via heavens gate
‚ùå Rejet√© (F1=0.0000, Prec=0.0000, Rec=0.0000, AUC=0.8864)
üîç Colonne : 64bits
‚úÖ Accept√© (F1=0.9087, Prec=0.9656, Rec=0.8581, AUC=0.9848)
üîç Colonne : PEB access
‚úÖ Accept√© (F1=0.7393, Prec=0.7768, Rec=0.7053, AUC=0.9519)
üîç Colonne : accept command line arguments
‚ùå Rejet√© (F1=0.5535, Prec=0.8151, Rec=0.4191, AUC=0.8902)
üîç Colonne : access the Windows event log
‚ùå Rejet√© (F1=0.4177, Prec=0.8560, Rec=0.2762, AUC=0.9277)
üîç Colonne : act as TCP client
‚ùå Rejet√© (F1=0.5045, Prec=0.9094, Rec=0.3491, AUC=0.9128)
üîç Colonne : allocate RW memory
‚ùå Rejet√© (F1=0.1890, Prec=0.7680, Rec=0.1077, AUC=0.8928)
üîç Colonne : allocate RWX memory
‚ùå Rejet√© (F1=0.2474, Prec=0.7143, Rec=0.1496, AUC=0.9161)
üîç Colonne : allocate memory
‚ùå Rejet√© (F1=0.5276, Prec=0.7661, Rec=0.4023, AUC=0.9101)
üîç Colonne : allocate or change RW memory
‚ùå Rejet√© (F1=0.3846, Prec=0.7211, Rec=0.2622, AUC=0.8892)
ü

In [25]:
y_pred_test["name"]=meta_test["name"]

In [27]:
# V√©rification de l'ordre des colonnes
if list(y_pred_test.columns) == list(meta_test.columns):
    print("‚úÖ L'ordre des colonnes est identique.")
else:
    print("‚ùå L'ordre des colonnes est diff√©rent.")
    # Affichage des diff√©rences pour investigation
    print("Colonnes y_pred_test :", list(y_pred_test.columns))
    print("Colonnes meta_test   :", list(meta_test.columns))
    


‚úÖ L'ordre des colonnes est identique.


In [26]:
# R√©ordonner les colonnes de y_pred_test pour qu'elles correspondent √† meta_test
y_pred_test = y_pred_test[meta_test.columns]


In [29]:

y_pred_test.to_excel("ebanutaya_hernya277777.xlsx", index=False)

In [30]:
y_pred_test

name  \
0     95ff2b3fd399984950293194a717d404bc4e9e3aa0296f...   
1     4216c07609bb5b89f32d9b559494848a0f5411d1e2d3cf...   
2     2518f84c015a5795bdb2597d580ab7df8e0bfa4b6543c6...   
3     243268c658456d9b8ec968c088c4f3c7cb976df92a4b99...   
4     533c09cb9b49c10494337d3eb7d2919c2c656b37f554fb...   
...                                                 ...   
2995  7f0a7ccc2abe273330885020f6dc5f116fe83535838b65...   
2996  a188ff24aec863479408cee54b337a2fce25b9372ba557...   
2997  11bf54ba134bfcf242e6c841ae56f44883cdf1b45fa24c...   
2998  7a8862b047f38a1f9ff37b6d1fa114f61bcb7ad6415bed...   
2999  5be7e227739f852689d1a0c7f4c42afe5e98a1090718e2...   

      64-bit execution via heavens gate  64bits  PEB access  \
0                                     0       1           0   
1                                     0       0           0   
2                                     0       1           0   
3                                     0       0           0   
4                                     0       1           0   
...                                 ...     ...         ...   
2995                                  0       0           0   
2996                                  0       1           0   
2997                                  0       0           0   
2998                                  0       0           0   
2999                                  0       1           0   

      accept command line arguments  access the Windows event log  \
0                                 1                             0   
1                                 0                             0   
2                                 1                             1   
3                                 0                             0   
4                                 1                             1   
...                             ...                           ...   
2995                              1                             0   
2996                              0                             0   
2997                              0                             0   
2998                              1                             0   
2999                              1                             0   

      act as TCP client  allocate RW memory  allocate RWX memory  \
0                     0                   0                    0   
1                     0                   0                    0   
2                     0                   0                    0   
3                     0                   0                    0   
4                     0                   0                    0   
...                 ...                 ...                  ...   
2995                  1                   0                    0   
2996                  0                   0                    0   
2997                  0                   0                    0   
2998                  1                   0                    0   
2999                  0                   0                    0   

      allocate memory  ...  winzip  wise  worm  write and execute a file  \
0                   0  ...       0     0     0                         0   
1                   0  ...       0     0     0                         0   
2                   1  ...       0     0     0                         0   
3                   0  ...       0     0     0                         0   
4                   1  ...       0     0     0                         0   
...               ...  ...     ...   ...   ...                       ...   
2995                0  ...       0     0     0                         0   
2996                0  ...       0     0     0                         0   
2997                0  ...       0     0     0                         0   
2998                1  ...       0     0     0                         1   
2999                1  ...       0     0     0                         1   

      write clipboard data

In [ ]:
ff=pd_read_exc